<a href="https://colab.research.google.com/github/yukinaga/automl/blob/main/section_4/01_automl_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# タイタニック号生存者の予測
AutoMLにより、タイタニック号の生存者を予測します。  
訓練済みのモデルによる予測結果は、csvファイルに保存して提出します。  

## PyCaretのインストール
AutoMLをサポートするライブラリ、PyCaretをバージョンを指定してインストールします。

In [ ]:
!pip install numpy==1.21.4 numba==0.53
!pip install pycaret==2.3.6
!pip install pandas-profiling==3.1.0

## Google Colabの設定
Google Colab環境でPyCaretのインタラクティブな要素を表示するためには、以下のコードを実行する必要があります。

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

## データの読み込み
以下のページからタイタニック号の乗客データをダウロードして、「train.csv」「test.csv」をノートブック環境にアップします。  
https://www.kaggle.com/c/titanic/data

以下のコードは、これらのデータを読み込みます。  
訓練データには乗客が生き残ったどうかを表す"Survived"の列がありますが、テストデータにはありません。  
訓練済みのモデルに、テストデータを入力して判定した結果を提出することになります。  


In [ ]:
import numpy as np
import pandas as pd

train_data = pd.read_csv("train.csv")  # 訓練データ
test_data = pd.read_csv("test.csv") # テストデータ

train_data.head()

## 環境の設定
PyCaretの環境を設定します。  
setup関数はPyCaretの環境を初期化しますが、PyCaretの他の関数を実行する前に呼び出す必要があります。      
データの型に問題が無ければ、空白を入力することで設定を完了することができます。  
  
今回のデータには一部欠損があるので、欠損データに対する対応を設定する必要があります。  
`numeric_imputation="mean"`により数値データの欠損には平均値があてがわれ、`categorical_imputation="mode"`によりカテゴリデータの欠損には最頻値があてがわれます。  

https://pycaret.readthedocs.io/en/latest/api/classification.html#pycaret.classification.setup


In [ ]:
from pycaret.classification import setup

clf = setup(data=train_data, target="Survived", session_id=123,  # 環境の初期化
            numeric_imputation="mean", categorical_imputation="mode")

## モデルの比較
様々なモデルを比較して、性能を評価します。  
compare_models関数は、ライブラリ内のすべてのモデルを使って訓練を行い、スコアを評価します。  

In [ ]:
from pycaret.classification import compare_models

best_model = compare_models()  # 全てのモデルを訓練し、評価する

わずか1行のコードで、15以上の機械学習モデルを訓練し、評価することができました。  
  
最もスコアの良いモデルの概要を表示します。


In [ ]:
print(best_model)

## モデルの作成
models関数により、全ての使用可能な機械学習モデルを確認することができます。

In [ ]:
from pycaret.classification import models

models()  # 機械学習モデルの一覧

create_model関数は、「交差検証」を用いて個別のモデルの訓練と評価を行います。  
今回は、先程最も精度が高かった「勾配ブースティング」による分類器（Gradient Boosting Classifier）のモデルを作成します。  
勾配ブースティングは弱いモデルが誤差に対して順次学習していく学習方法で、一つの強いモデルを作らず、複数のモデルを組み合わせて強力なモデルとします。  
この弱いモデルとして、決定木がよく使われます。

In [ ]:
from pycaret.classification import create_model

gbc = create_model("gbc")  # 勾配ブースティングのモデルを作成

訓練済みモデルの概要を表示します。  

In [ ]:
print(gbc)

## ハイパーパラメータの調整
tune_model関数を使用し、ハイパーパラメータを最適化します。

In [ ]:
from pycaret.classification import tune_model

tuned_gbc = tune_model(gbc)  # ハイパーパラメータの調整

ハイパーパラメータを調整済みの、モデルの概要を表示します。  

In [ ]:
print(tuned_gbc)

## モデルを評価する
plot_model関数を使い、各特徴量の重要度をプロットします。

In [ ]:
from pycaret.classification import plot_model

plot_model(tuned_gbc, plot="feature")  # 各特徴の重要度をプロット

## 本番用のモデルを作成
`finalize_model`関数により全ての訓練データを使ってモデルを訓練し、本番用のモデルを作成します。   

In [ ]:
from pycaret.classification import finalize_model

final_gbc = finalize_model(tuned_gbc)
print(final_gbc)

## 提出用のデータを作成
テストデータを使って予測を行います。  
予測結果には、分類されるクラスを表す「Label」列と、そのクラスに含まれる確率を表す「Score」ラベルが含まれます。

In [ ]:
from pycaret.classification import predict_model

test_pred = predict_model(final_gbc, data=test_data)  # 予測
test_pred.head()

形式を整えた上で、Kaggleに提出するためのcsvファイルを保存します。

In [ ]:
# 形式を整える
subm_data = test_pred[["PassengerId", "Label"]]  # 列を抜き出す
subm_data = subm_data.rename(columns={"Label" : "Survived"})  # 列名の変更

# 提出用のcsvファイルを保存
subm_data.to_csv("submission_titanic.csv", index=False)

subm_data